# Premier league database

Mediante este proyecto seremos capaces de montar una base de datos con los resultados de la premier league, para ello, deberemos:

* Scrappear una web de terceros para obtener la información
* Procesar y limpiar dichos datos para que nos sean útiles
* Diseñar y montar una base de datos apropiada para nuestros datos
* Juntarlo todo

## La web

En primer lugar debemos encontrar una fuente para obtener nuestros datos. Una conocida página para consultar este tipo de resultados es https://www.jornadaperfecta.com/. En el caso de la Premier League, los resultados pueden ser encontrados en https://www.jornadaperfecta.com/premier/puntos/, y más específicamente, jornada por jornada, en https://www.jornadaperfecta.com/premier/puntos/?idJornada=1. Como se puede apreciar, la estructura de carpetas de la web es idónea para nuestra tarea, dado que bastará con ir sustituyendo el último número de la URL para obtener los datos de las sucesivas jornadas.

Se puede observar que las jornadas se organizan en cajas, cada una de las cuales es un partido, el cual contiene dos equipos y estos a su vez todos los jugadores involucrados, con sus nombres, puntos, tarjetas y goles.
Si inspeccionamos el elemento que corresponde a la "caja" de cada partido (_div class="col-6-12"_) nos encontramos:

<br>
<div>
<img src="img/2.png" width="1000">
</div>
<br>

Se pueden identificar ya las etiquetas que nos darán los nombres de los equipos emparejados (_meta itemprop="name"_) así como la fecha de inicio del encuentro (_time itemprop="startDate"_). 
Además, podemos ver que dentro de esta existen dos contenedores inferiores (_div class="col-6-12 mobile-col-6-12"_), uno para cada uno de los dos equipos. 
Dentro de estos, a su vez, podemos observar que cada jugador se registra en un objeto propio (_div class="puntos-jugador"_). Inspeccionando este último elemento hallamos toda la lógica del registro de atributos de cada jugador:

<br>
<div>
<img src="img/3b.png" width="1000">
</div>
<br>

Encontramos así todas las etiquetas relevantes para nuestro proyecto:
* La imagen (y el ID interno): _meta itemprop="image"_
* El nombre (y el link interno): _meta itemprop="name"_
* La posición: _div class="jugador-posicion pt"_  (pt/df/mc/dl)
* Los puntos obtenidos: _div class="puntos-jugador-puntuacion *"_

Ya podemos empezar a recabar algunos datos:
<br><br>


In [4]:
from bs4 import BeautifulSoup
import requests

link = 'https://www.jornadaperfecta.com/premier/puntos/?idJornada={}'.format(1)
webpage_response = requests.get(link)
webpage = webpage_response.content
soup = BeautifulSoup(webpage, "html.parser")

# Buscamos la clase "puntos-partido" pero nos habría dado igual buscar "col-6-12"
partidos = soup.find_all("div", {"class": "puntos-partido"})

# Contamos cuántos partidos hemos conseguido listar, deberían ser 10:
print("Partidos listados: ",len(partidos))

Partidos listados:  10


In [5]:
# Ahora consultamos el contenido de nuestra lista de partidos, veamos el comienzo del primer elemento:
print(str(partidos[0])[:500])

<div class="puntos-partido" itemscope="" itemtype="http://schema.org/SportsEvent"><time content="2023-08-11T21:00:00+02:00" itemprop="startDate"></time><meta content="Burnley - Manchester City" itemprop="name"/><div itemprop="location" itemscope="" itemtype="http://schema.org/StadiumOrArena" style="display:none"><meta content="Turf Moor" itemprop="name"/><meta content="Turf Moor, 52-56 Harry Potts Way, Burnley BB10 4BX, Reino Unido" itemprop="address"/><meta content="/assets/images/estadios/64.j



Se puede encontrar a simple vista el nombre de los dos equipos que se enfrentan en este primer encuentro de la primera jornada: Burnley - Manchester City.
Vamos a obtener la lista completa:


In [6]:
lista_emparejamientos = []        
for partido in partidos:
    lista_emparejamientos.append(str(partido.find("meta", itemprop="name")).split('"')[1].split(" - "))
    lista_emparejamientos_separados = [x for l in lista_emparejamientos for x in l]
    
print(lista_emparejamientos)

[['Burnley', 'Manchester City'], ['Arsenal', 'Nottingham Forest'], ['Sheffield United', 'Crystal Palace'], ['Everton', 'Fulham'], ['Brighton &amp; Hove Albion', 'Luton Town'], ['AFC Bournemouth', 'West Ham United'], ['Newcastle', 'Aston Villa'], ['FC Brentford', 'Tottenham Hotspur'], ['Chelsea', 'Liverpool'], ['Manchester United', 'Wolverhampton']]


In [7]:
# El resto de proceso es igual de directo, podemos hallar la lista de equipos buscando los contenedores que identificamos 
# anteriormente:

equipos = soup.find_all("div", {"class": "col-6-12 mobile-col-6-12"})

# Y con ello hallar el resto de información relevante de cada partido:

lista_jugadores = []
for equipo in equipos:

    #Utilizamos BeautifulSoup para buscar la tabla de cada jugador:
    jugadores = equipo.find_all("div", {"class": "puntos-jugador"})

    #Nos metemos en cada jugador y obtenemos los datos relevantes:
    for jugador in jugadores:

        posicion = str(jugador.find("div", {"class": "jugador-posicion"}).text)
        puntuacion = str(jugador.find("div", {"class": "puntos-jugador-puntuacion"}).text)
        nombre = str(jugador.find("a", itemprop="url").text)
        
        lista_jugadores.append((nombre, posicion, puntuacion))

print(lista_jugadores[1:15])

[('Ameen Al-Dakhil', 'DF', '2'), ('Connor Roberts', 'DF', '2'), ("Dara O'Shea", 'DF', '2'), ('Jordan Beyer', 'DF', '4'), ('Vitinho', 'DF', '1'), ('Josh Cullen', 'MC', '2'), ('Sander Berge', 'MC', '2'), ('Luca Koleosho', 'DL', '2'), ('Lyle Foster', 'DL', '2'), ('Zeki Amdouni', 'DL', '4'), ('Anass Zaroury', 'DL', '2'), ('Jacob Bruun Larsen', 'DL', '3'), ('Manuel Hedilazio', 'DL', '-1'), ('Josh Brownhill', 'MC', '2')]



Definimos un par de funciones para realizar este proceso de forma automática para cualquier jornada:

* Una primera función que scrapee la web y devuelva las strings correspondientes a los bloques de código del contenedor de cada partido y de cada equipo

* Una segunda función que extraiga de estos strings la información relevante y la almacene provisionalmente de forma ordenada


In [8]:
def scrapear(j):
    link = 'https://www.jornadaperfecta.com/premier/puntos/?idJornada={}'.format(j)
    webpage_response = requests.get(link)
    webpage = webpage_response.content
    soup = BeautifulSoup(webpage, "html.parser")
          
    partidos = soup.find_all("div", {"class": "puntos-partido"})
    equipos = soup.find_all("div", {"class": "col-6-12 mobile-col-6-12"})
    
    return(partidos, equipos)

In [9]:
def recabar(j):
    
    partidos, equipos = scrapear(j)
        
    lista_clubes = []
    lista_encuentros =  []
    
    for partido in partidos:

        
        marcador = (partido.find_all("span", {"class": "puntos-resultado-marcador"})[0].text, partido.find_all("span", {"class": "puntos-resultado-marcador"})[1].text)
 
        local = partido.find("div", itemprop="homeTeam").find(itemprop="name")["content"]
        visitante = partido.find("div", itemprop="awayTeam").find(itemprop="name")["content"]
        
        lista_clubes.extend([local, visitante])
        lista_encuentros.append((j, local, visitante, marcador))
    
    # Inicializamos algunas listas y la variable n para ir alternando oponentes:
    lista_jugadores=[]
    n=0
    
    # Entramos en cada equipo:
    for equipo in equipos:
        
        # Utilizamos BeautifulSoup para buscar la tabla de cada jugador:
        jugadores = equipo.find_all("div", {"class": "puntos-jugador"})
        
        # Entramos en cada jugador:
        for jugador in jugadores:
            
            # Utilizamos BeautifulSoup para buscar las etiquetas relevantes:
            posicion = str(jugador.find("div", {"class": "jugador-posicion"}).text)
            puntuacion = str(jugador.find("div", {"class": "puntos-jugador-puntuacion"}).text)
            nombre = str(jugador.find("a", itemprop="url").text)
            eventos = [icon.get("title") for icon in jugador.find_all("i", {"class": "icon"})]
  
            club = lista_clubes[equipos.index(equipo)]
            oponente = lista_clubes[equipos.index(equipo)+(-1)**n]      
            localidad = "local" if lista_clubes.index(club) % 2 == 0 else "visitante"
    
            # Añadimos el registro del jugador a la lista de jugadores:
            lista_jugadores.append([j, nombre, posicion, puntuacion, club, oponente, localidad, eventos])

        n+=1   # Este índice sirve para alternar oponentes
    

    return(lista_jugadores, lista_encuentros, lista_clubes)

In [10]:
# Obtenemos los resulados para la primera jornada:
jugadores, encuentros, equipos = recabar(1)

In [12]:
# Formateamos en un dataframe los jugadores y visualizamos algunas filas:
import pandas as pd

jugadores_df = pd.DataFrame(jugadores, columns=["Jornada", "Nombre", "Posición", "Puntos", "Equipo", "Oponente", "Juega", "Eventos"])

jugadores_df.sample(10)

,Jornada,Nombre,Posición,Puntos,Equipo,Oponente,Juega,Eventos
119,1,Groß,MC,11,Brighton & Hove Albion,Luton Town,local,[]
128,1,Evan Ferguson,DL,11,Brighton & Hove Albion,Luton Town,local,[Gol]
87,1,Odsonne Edouard,DL,13,Crystal Palace,Sheffield United,visitante,[Gol]
278,1,Rashford,DL,4,Manchester United,Wolverhampton,local,[]
161,1,James Hill,DF,3,AFC Bournemouth,West Ham United,local,[]
95,1,Amadou Onana,MC,8,Everton,Fulham,local,[]
49,1,Joe Worrall,DF,3,Nottingham Forest,Arsenal,visitante,[]
268,1,André Onana,PT,10,Manchester United,Wolverhampton,local,[]
37,1,Rice,MC,7,Arsenal,Nottingham Forest,local,[]
176,1,Maxwel Cornet,DL,0,West Ham United,AFC Bournemouth,visitante,[]


In [14]:
# Formateamos en un dataframe los encuentros y visualizamos algunas filas:

encuentros_df = pd.DataFrame(encuentros,  columns =['Jornada','Equipo local','Equipo visitante', 'Resultado'])

encuentros_df.head(10)

,Jornada,Equipo local,Equipo visitante,Resultado
0,1,Burnley,Manchester City,"(0, 3)"
1,1,Arsenal,Nottingham Forest,"(2, 1)"
2,1,Sheffield United,Crystal Palace,"(0, 1)"
3,1,Everton,Fulham,"(0, 1)"
4,1,Brighton & Hove Albion,Luton Town,"(4, 1)"
5,1,AFC Bournemouth,West Ham United,"(1, 1)"
6,1,Newcastle,Aston Villa,"(5, 1)"
7,1,FC Brentford,Tottenham Hotspur,"(2, 2)"
8,1,Chelsea,Liverpool,"(1, 1)"
9,1,Manchester United,Wolverhampton,"(1, 0)"



Ahora disponemos de una manera relativamente sencilla de generar dataframes con toda la información que hemos considerado relevante. El siguiente objetivo es construir una base de datos SQL que pueda almacenar toda esta información. Para ello, diseñamos el siguiente esquema:

<br>
<div>
<img src="img/4c.png" width="800">
</div>
<br>



Para llevarlo a cabo, instalamos PostgreSQL y Postbird (para visualizar la base de datos). Después instalamos la librería psycopg2 para poder ejecutar SQL desde Python. A continuación no hay más que definir una función para comunicarnos con la base de datos:


In [15]:
import psycopg2

def query(peticion, valores=False):
    # Conecta a una base de datos existente
    conn = psycopg2.connect(dbname="postgres", user="postgres", password="postgres", port='5432')
    
    # Inicializa un cursor para realizar las operaciones
    cur = conn.cursor()
    
    # Ejecuta la petición
    if not valores:
        cur.execute(peticion)
    else:
        cur.execute(peticion, valores)
    
    # Hace que los cambios sean persistentes
    conn.commit()
    
    # Termina la comunicación con la base de datos
    cur.close()
    conn.close()


Y otra para crear nuestras tablas, pasándole las querys en forma de strings:


In [29]:
def create_tables():
    
    first_table = """
    CREATE TABLE equipos (
      id integer PRIMARY KEY,
      nombre varchar UNIQUE NOT NULL
    
    );
    """
    
    second_table = """
    CREATE TABLE jugadores (
      id integer PRIMARY KEY,
      nombre varchar UNIQUE NOT NULL,
      equipo varchar REFERENCES equipos(nombre),
      posicion varchar
    
    );
    """
    
    third_table = """
    CREATE TABLE partidos (
      id integer PRIMARY KEY,
      jornada integer,
      local varchar REFERENCES equipos(nombre),
      visitante varchar REFERENCES equipos(nombre),
      goles_local integer,
      goles_visitante integer
    
    );
    """
    
    fourth_table = """
    CREATE TABLE resultados (
      id integer PRIMARY KEY,
      jugador varchar REFERENCES jugadores(nombre) NOT NULL,
      jornada integer,
      puntos integer,
      goles integer,
      asistencias integer,
      tarjetas_amarillas integer,
      tarjetas_rojas integer
    
    );
    """
    
    query(first_table)
    query(second_table)
    query(third_table)
    query(fourth_table)
    
create_tables()


Por último, podemos recorrer cada dataframe con un bucle e ir añadiendo los registros a nuestra base de datos. Para añadir cada registro primero debemos definir las strings que vamos a pasarle a psycopg2, y después darle forma al contenido. He aquí algunos ejemplos:


In [30]:
equipos_string = """
INSERT INTO equipos (id, nombre)
VALUES (%s, %s);
"""
jugadores_string = """
INSERT INTO jugadores (id, nombre, equipo, posicion)
VALUES (%s, %s, %s, %s);
"""
resultados_string = """
INSERT INTO resultados (id, jugador, jornada, puntos, goles, asistencias, tarjetas_amarillas, tarjetas_rojas)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
"""
partidos_string = """
INSERT INTO partidos (id, jornada, local, visitante, goles_local, goles_visitante)
VALUES (%s, %s, %s, %s, %s, %s);
"""

valores_equipos_1 = (1, "Burnley")
valores_equipos_2 = (2, "Manchester City")
valores_jugadores = (1, "James Trafford", "Burnley", "PT")
valores_resultados = (1, "James Trafford", 1, 1, 7, 0, 0, 0)
valores_partidos = (1, 1, "Burnley", "Manchester City", 0, 3)

query(equipos_string, valores_equipos_1)
query(equipos_string, valores_equipos_2)
query(jugadores_string, valores_jugadores)
query(resultados_string, valores_resultados)
query(partidos_string, valores_partidos)